In [3]:
class MyClass:
    x = 5


In [5]:
p1 = MyClass()
print(p1.x)

5


In [21]:
class test:
    def __init__(self, num):
        self.x = num
    def add(self, num):
        self.x += num
    def sub(self, num):
        self.x -= num
    def mul(self, num):
        self.x *= num
    def div(self, num):
        self.x /= num
    def get(a):
        return a.x

c = test(5)

c.add(2)

c.get()


7

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split


In [33]:

class modelManager:
    """
    This is a class to manage the different models, to have the same train test split when using multiple models
    Takes the full dataframe, X and target y as input
    """
    def __init__(self, data, X, y) -> None:
        self.data = data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    """
    This functions will return fitted models with the train test split
    """
    def decisionTree(self) -> dict:
        from sklearn.tree import DecisionTreeClassifier
        model = DecisionTreeClassifier()
        model.fit(self.X_train, self.y_train)
        return {"model": model, "X_train": self.X_train, "X_test": self.X_test, "y_train": self.y_train, "y_test": self.y_test}
    def naiveBayes(self) -> dict:
        # make a naive bayes model
        from sklearn.naive_bayes import GaussianNB
        model = GaussianNB()
        model.fit(self.X_train, self.y_train)
        return {"model": model, "X_train": self.X_train, "X_test": self.X_test, "y_train": self.y_train, "y_test": self.y_test}
    def randomForest(self) -> dict:
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier(n_estimators=100)
        model.fit(self.X_train, self.y_train)
        return {"model": model, "X_train": self.X_train, "X_test": self.X_test, "y_train": self.y_train, "y_test": self.y_test}
    def svm(self) -> dict:
        from sklearn.svm import SVC
        model = SVC(gamma='auto')
        model.fit(self.X_train, self.y_train)
        return {"model": model, "X_train": self.X_train, "X_test": self.X_test, "y_train": self.y_train, "y_test": self.y_test}

In [25]:
# notes
"""
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=100, centers=2, n_features=2, random_state=42)
print(X.shape, y.shape)
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
model = LogisticRegression(solver='lbfgs')
# fit model
model.fit(X, y)
# make predictions
yhat = model.predict(X)
# evaluate predictions
acc = accuracy_score(y, yhat)
print(acc)
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
# create the inputs and outputs
X, y = make_blobs(n_samples=1000, centers=2, n_features=2, random_state=2)
# define model
model = LogisticRegression(solver='lbfgs')
# fit model
model.fit(X, y)
# define input
new_input = [[2.12309797, -1.41131072]]
# get prediction for new input
new_output = model.predict(new_input)
# summarize input and output
print(new_input, new_output)
"""

(100, 2) (100,)
